In [1]:
# The Bootstrap
# k-Fold Cross-Validation
# Leave-One-Out Cross-Validation
# 5.3.1
# The validation set approch
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn.linear_model as skl_lm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, classification_report, precision_score, mean_squared_error
from sklearn import preprocessing
from sklearn import neighbors
from sklearn.model_selection import train_test_split

import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn import datasets 
# https://realpython.com/python-csv/
# Read CSV file python
import pandas
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
import statsmodels.api as sm
from statsmodels.graphics.regressionplots import *
from sklearn import datasets, linear_model


In [2]:
df = pandas.read_csv('Dataset/Portfolio.csv', header = 0)
df.shape
#print(df)

(100, 3)

In [3]:
df.head()

,Unnamed: 0,X,Y
0,1,-0.895251,-0.234924
1,2,-1.562454,-0.885176
2,3,-0.417090,0.271888
3,4,1.044356,-0.734198
4,5,-0.315568,0.841983


# Estimating the Accuracy of a Statistic of Interest

In [4]:
def alpha_fn(data, index):
    X = data.X[index]
    Y = data.Y[index]
    return (np.var(Y) - np.cov(X,Y)[0,1])/(np.var(X) + np.var(Y) - 2 * np.cov(X, Y)[0,1])

In [5]:
alpha_fn(df, list(range(0,100)))

0.5766511516104118

In [6]:
np.random.seed(17) 
np.sort(np.random.choice(range(0, 100), size=100, replace=True))
alpha_fn(df, np.random.choice(range(0, 100), size=100, replace=True))

0.6466464608560455

In [7]:
def boot_python(data, input_function, iteration):
    n = df.shape[0]
    idx = np.random.randint(0, n, (iteration, n))
    stat = np.zeros(iteration)
    for i in range(len(idx)):
        stat[i] = input_function(data, idx[i])
    
    return {'Mean': np.mean(stat), 'STD': np.std(stat)}

In [8]:
boot_python(df, alpha_fn, 1000)

{'Mean': 0.5800666255791905, 'STD': 0.09147655756203628}

# Estimating the Accuracy of a Linear Regression Model

In [9]:
Auto = pandas.read_csv('Dataset/Auto.csv', na_values = '?', header = 0).dropna().reset_index(drop = True)

In [10]:
def alpha_function_auto(data, index):
    X = data.horsepower[index]
    Y = data.mpg[index]
    return (np.var(Y) - np.cov(X,Y)[0,1])/(np.var(X) + np.var(Y) - 2 * np.cov(X, Y)[0,1])

In [11]:
def boot_python_auto(data, input_function, iteration):
    n = Auto.shape[0]
    idx = np.random.randint(0, n, (iteration, n))
    stat = np.zeros(iteration)
    for i in range(len(idx)):
        stat[i] = input_function(data, idx[i])
    
    return {'Mean': np.mean(stat), 'STD': np.std(stat)}

In [12]:
# Bootstrap standard error 
res = boot_python_auto(Auto, alpha_function_auto, 1000)
print(res)

{'Mean': 0.14710206859431882, 'STD': 0.006088123222815259}


In [13]:
Auto.describe()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin
count,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000
mean,23.445918,5.471939,194.411990,104.469388,2977.584184,15.541327,75.979592,1.576531
std,7.805007,1.705783,104.644004,38.491160,849.402560,2.758864,3.683737,0.805518
min,9.000000,3.000000,68.000000,46.000000,1613.000000,8.000000,70.000000,1.000000
25%,17.000000,4.000000,105.000000,75.000000,2225.250000,13.775000,73.000000,1.000000
50%,22.750000,4.000000,151.000000,93.500000,2803.500000,15.500000,76.000000,1.000000
75%,29.000000,8.000000,275.750000,126.000000,3614.750000,17.025000,79.000000,2.000000
max,46.600000,8.000000,455.000000,230.000000,5140.000000,24.800000,82.000000,3.000000


In [14]:
from sklearn.utils import resample

In [15]:
lm = LinearRegression()
x = Auto['horsepower'].values.reshape(-1,1)
y = Auto['mpg']

clf = lm.fit(x,y)

In [16]:
# Estimates: Slope (horsepower) and intercept
print('Coeffient:' + str(clf.coef_), 'Intercept: ' + str(clf.intercept_))

Coeffient:[-0.15784473] Intercept: 39.93586102117047


In [17]:
# Estimates: Slope (horsepower) and intercept with replacements and 1000 bootstrap samples
x_samp, y_samp = resample(x, y, n_samples=1000)
clf_new = lm.fit(x_samp, y_samp)
print('Coeffient:' + str(clf_new.coef_), 'Intercept: ' + str(clf_new.intercept_))

Coeffient:[-0.16319052] Intercept: 40.82564711094224


In [18]:
# theoretical standard error
import scipy.stats
xx = Auto['horsepower']
slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(xx,y)
print(std_err)

0.006445500517685033
